In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install -r requirements.txt

In [ ]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [ ]:
!pwd

In [ ]:
%mkdir Road_Sign_Dataset
%cd Road_Sign_Dataset
!mkdir annotations
!mkdir images

In [ ]:
%cd /kaggle/working/Road_Sign_Dataset
!ls

In [ ]:
#!rm -R Road_Sign_Dataset

In [ ]:
!cp -R /kaggle/input/tbx-11/TBX11K/annotations/xml/* /kaggle/working/Road_Sign_Dataset/annotations
!cp -R /kaggle/input/tbx-11/TBX11K/imgs/tb/* /kaggle/working/Road_Sign_Dataset/images

In [ ]:
!pwd
!ls

In [ ]:
# Assuming you're in the data folder
!cat /kaggle/working/Road_Sign_Dataset/annotations/tb0004.xml

In [ ]:
root = ET.parse('/kaggle/working/Road_Sign_Dataset/annotations/tb0026.xml').getroot()
root

In [ ]:
# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text + '.xml'
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [ ]:
import os
os.remove('/kaggle/working/Road_Sign_Dataset/annotations/tb1199.xml')
os.remove('/kaggle/working/Road_Sign_Dataset/images/tb1199.png')

In [ ]:
print(extract_info_from_xml('/kaggle/working/Road_Sign_Dataset/annotations/tb0004.xml'))

In [ ]:
{'bboxes': [{'class': 'trafficlight', 'xmin': 20, 'ymin': 109, 'xmax': 81, 'ymax': 237}, {'class': 'trafficlight', 'xmin': 116, 'ymin': 162, 'xmax': 163, 'ymax': 272}, {'class': 'trafficlight', 'xmin': 189, 'ymin': 189, 'xmax': 233, 'ymax': 295}], 'filename': 'road4.png', 'image_size': (267, 400, 3)}


In [ ]:
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"ActiveTuberculosis": 0,
                           "ObsoletePulmonaryTuberculosis": 1}
                            #,
                           #"speedlimit": 2,
                           #"crosswalk": 3}

# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_c = 3
        image_w, image_h = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
    
    #print(info_dict["filename"])
    # Name of the file which we have to save 
    save_file_name = os.path.join("annotations", info_dict["filename"].replace(".xml", ".txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [ ]:
#annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "xml"]
#annotations.sort()
#annotations

In [ ]:
# Get the annotations
annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "xml"]
annotations.sort()

# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    #print(info_dict)
    convert_to_yolov5(info_dict)
annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]

In [ ]:
#{'bboxes': [{'class': 'ObsoletePulmonaryTuberculosis', 'xmin': 1446, 'ymin': 250, 'xmax': 2007, 'ymax': 1017}, {'class': 'ObsoletePulmonaryTuberculosis', 'xmin': 618, 'ymin': 391, 'xmax': 1177, 'ymax': 1013}], 'filename': 'tb0003', 'image_size': (2840, 2827)}

# Testing the annotations

In [ ]:
#annotations

In [ ]:
annotation_file = random.choice(annotations)

In [ ]:
random.seed(0)

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)))
        
        plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))])
    
    plt.imshow(np.array(image))
    plt.show()

# Get any random annotation file 
annotation_file = random.choice(annotations)
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]

#Get the corresponding image file
image_file = annotation_file.replace("annotations", "images").replace("txt", "png")
assert os.path.exists(image_file)

#Load the image
image = Image.open(image_file)

#Plot the Bounding Box
plot_bounding_box(image, annotation_list)


# Partition the dataset

In [ ]:
annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]
#annotations

In [ ]:
# Read images and annotations
images = [os.path.join('images', x) for x in os.listdir('images')]
annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits 
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)


In [ ]:
!pwd

In [ ]:
!mkdir images/train 
!mkdir images/val 
!mkdir images/test 

!mkdir annotations/train 
!mkdir annotations/val 
!mkdir annotations/test

In [ ]:
len(os.listdir('/kaggle/working/Road_Sign_Dataset/images/train'))
#len(os.listdir('/kaggle/working/Road_Sign_Dataset/images/test'))
#len(os.listdir('/kaggle/working/Road_Sign_Dataset/images/val'))

In [ ]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, '/kaggle/working/Road_Sign_Dataset/images/train')
move_files_to_folder(val_images, '/kaggle/working/Road_Sign_Dataset/images/val/')
move_files_to_folder(test_images, '/kaggle/working/Road_Sign_Dataset/images/test/')
move_files_to_folder(train_annotations, '/kaggle/working/Road_Sign_Dataset/annotations/train/')
move_files_to_folder(val_annotations, '/kaggle/working/Road_Sign_Dataset/annotations/val/')
move_files_to_folder(test_annotations, '/kaggle/working/Road_Sign_Dataset/annotations/test/')
!mv annotations labels
%cd ../

In [ ]:
%cd /kaggle/working/Road_Sign_Dataset/
!ls

In [ ]:
len(os.listdir('/kaggle/working/Road_Sign_Dataset/labels/train/'))


In [ ]:
!wget -P data/ https://gist.githubusercontent.com/ayooshkathuria/bcf7e3c929cbad445439c506dba6198d/raw/f437350c0c17c4eaa1e8657a5cb836e65d8aa08a/road_sign_data.yaml


# INSTALL

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git &> /dev/null

In [ ]:
%cd yolov7

In [ ]:
!pip install -r requirements.txt &> /dev/null

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-e6e.pt

In [ ]:
!wget https://raw.githubusercontent.com/tkeldenich/How_to_use_YOLOv7_Tutorial/main/man_cafe.jpg

In [ ]:
from PIL import Image
im = Image.open('man_cafe.jpg')
im

In [ ]:
!python detect.py --weights yolov7-e6e.pt --source ./man_cafe.jpg

In [ ]:
im_res1 = Image.open('./runs/detect/exp/man_cafe.jpg')
im_res1

In [ ]:
!python detect.py --weights yolov7-e6e.pt --conf 0.60 --img-size 640 --source ./man_cafe.jpg

In [ ]:
im_res2 = Image.open('./runs/detect/exp2/man_cafe.jpg')
im_res2

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt

In [ ]:
!python detect.py --weights yolov7-tiny.pt --conf 0.25 --img-size 640 --source ./man_cafe.jpg

In [ ]:
im_res3 = Image.open('./runs/detect/exp3/man_cafe.jpg')
im_res3

In [ ]:
!cp -R /kaggle/working/Road_Sign_Dataset/data/road_sign_data.yaml /kaggle/working/Road_Sign_Dataset/yolov7/data/

In [ ]:
!pwd

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
!pwd

In [ ]:
yamltext = """
train: /kaggle/working/Road_Sign_Dataset/images/train/ 
val:  /kaggle/working/Road_Sign_Dataset/images/val/
test: /kaggle/working/Road_Sign_Dataset/images/test/

# number of classes
nc: 2

# class names
names: ["ActiveTuberculosis","ObsoletePulmonaryTuberculosis"]

"""

with open("/kaggle/working/Road_Sign_Dataset/yolov7/data/road_sign_data.yaml", 'w') as file:
    file.write(yamltext)

#%cat /kaggle/working/data/data.yaml


!cat /kaggle/working/Road_Sign_Dataset/yolov7/data/road_sign_data.yaml

In [ ]:
%cd Road_Sign_Dataset/yolov7

In [ ]:
len(os.listdir('/kaggle/working/Road_Sign_Dataset/images/train/'))

# Train for 60 Epochs

In [ ]:
!python train.py --img-size 512 --cfg cfg/training/yolov7.yaml --hyp data/hyp.scratch.custom.yaml --batch 8 --epochs 10 --data data/road_sign_data.yaml --weights yolov7-e6e.pt --workers 24 --name yolo_road_det

# Train for 120 Epochs

In [ ]:
!python train.py --img-size 512 --cfg cfg/training/yolov7.yaml --hyp data/hyp.scratch.custom.yaml --batch 8 --epochs 130 --data data/road_sign_data.yaml --weights yolov7-e6e.pt --workers 24 --name yolo_det31Aug_130

In [ ]:
!pwd

# Inference for Train Run of 60 Epochs

In [ ]:
!python detect.py --source /kaggle/working/Road_Sign_Dataset/images/test/ --weights runs/train/yolo_road_det5/weights/best.pt --conf 0.05 --name yolo_road_det5

In [ ]:
detections_dir = "runs/detect/yolo_road_det5/"
detection_images = [os.path.join(detections_dir, x) for x in os.listdir(detections_dir)]

random_detection_image = Image.open(random.choice(detection_images))
plt.imshow(np.array(random_detection_image))

# No weights

In [ ]:
!python train.py --img-size 512 --cfg cfg/training/yolov7.yaml --hyp data/hyp.scratch.custom.yaml --batch 8 --epochs 10 --data data/road_sign_data.yaml --weights '' --workers 24 --name yolo_det31Aug_noweights


#yolov7-e6e.pt 

In [ ]:
!python test.py --weights runs/train/yolo_road_det3/weights/best.pt --data road_sign_data.yaml --task test --name yolo_det

# Inference for 130 Epochs

In [ ]:
!python detect.py --source /kaggle/working/Road_Sign_Dataset/images/test/ --weights runs/train/yolo_det31Aug_130/weights/best.pt --conf 0.05 --name yolo_road_det5



In [ ]:
detections_dir = "runs/detect/yolo_road_det52/"
detection_images = [os.path.join(detections_dir, x) for x in os.listdir(detections_dir)]

random_detection_image = Image.open(random.choice(detection_images))
plt.imshow(np.array(random_detection_image))